In [54]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from math import sqrt
import datetime

from itertools import product

from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer

# Functions

In [55]:
def get_all_combinations(df):
    
    all_combinations = []
    for dbn in df['date_block_num'].unique():
        
        unique_shops = df[df.date_block_num == dbn].shop_id.unique()
        unique_items = df[df.date_block_num == dbn].item_id.unique()
        
        all_combinations.append(np.array(list(product(unique_shops, unique_items, [dbn]))))
    
    # convert list into a single data frame 
    all_combinations = pd.DataFrame(np.vstack(all_combinations), columns = ['shop_id', 'item_id', 'date_block_num'], dtype=np.int32)

    return(all_combinations)
    


def get_shop_item_level_aggregates(df):
    
    shop_item_date_level = df.groupby(['date_block_num','shop_id','item_id'])\
        .agg({
            'item_cnt_day' : np.sum,
            'item_price'   : np.mean
        })\
        .reset_index()\
        .rename(columns = {'item_cnt_day': 'item_cnt_month'})
    
    return(shop_item_date_level)



def get_shop_level_aggregates(df):

    shop_level = df\
        .assign(
            SHOP_mean_item_price   = df.item_price,
            SHOP_sum_item_cnt_day  = df.item_cnt_day,
            SHOP_mean_item_cnt_day = df.item_cnt_day
            )\
        .groupby(['date_block_num', 'shop_id'])\
        .agg({
            'SHOP_mean_item_price'   : np.mean, # i.e. mean(sales.item_price) etc.
            'SHOP_sum_item_cnt_day'  : np.sum,
            'SHOP_mean_item_cnt_day' : np.mean       
        })\
        .reset_index()
           
    return shop_level
 
 
def get_item_level_aggregates(df):    
    
    item_level = df\
        .assign(
            ITEM_mean_item_price   = df.item_price,
            ITEM_sum_item_cnt_day  = df.item_cnt_day,
            ITEM_mean_item_cnt_day = df.item_cnt_day
            )\
        .groupby(['date_block_num', 'item_id'])\
        .agg({
            'ITEM_mean_item_price'   : np.mean,
            'ITEM_sum_item_cnt_day'  : np.sum,
            'ITEM_mean_item_cnt_day' : np.mean       
        })\
        .reset_index()
 
    return item_level
    
 
def get_item_category_level_aggregates(df):      
           
    item_category_level = df\
        .assign(
            ITEM_CATEGORY_mean_item_price   = df.item_price,
            ITEM_CATEGORY_sum_item_cnt_day  = df.item_cnt_day,
            ITEM_CATEGORY_mean_item_cnt_day = df.item_cnt_day
            )\
        .groupby(['date_block_num', 'item_category_id'])\
        .agg({
            'ITEM_CATEGORY_mean_item_price'   : np.mean,
            'ITEM_CATEGORY_sum_item_cnt_day'  : np.sum,
            'ITEM_CATEGORY_mean_item_cnt_day' : np.mean       
        })\
        .reset_index()

    return item_category_level 

    
   
    
def create_lags(df, lags, row_identifier_cols, to_lag_cols, ignore_cols):

    # for each lag, we create a new version of the data, with date_block_num increased by the lag, so
    # that we can join it to the corresponding 'future' row of the original data

    for lag in lags:
    
        # date_block_num is increased by the lag, so we can join to corresponding 'future' row
        lagged_df = df[to_lag_cols + row_identifier_cols] # this avoids columns created by previous lags
        lagged_df[['date_block_num']] += lag
        
        # give the new columns informative names
        col_suffix = '_lag' + str(lag)
        lagged_df.columns = [col + col_suffix if col not in row_identifier_cols + ignore_cols else col for col in lagged_df.columns ]
        
        # join to the main data, matching each row to its 'future' counterpart
        df = pd.merge(df, lagged_df, on=row_identifier_cols, how='left')

    return(df)    
    
    
    
    
def encode_category(df, col_to_encode):

    # (note that one-hot column numbers don't match original id numbers)
    encoded_cols = pd.get_dummies(df[col_to_encode], prefix=col_to_encode )
    df = pd.concat([encoded_cols, df.reset_index()], axis=1)
    
    return(df)

# Load the data

In [56]:
# Get the original data:
# downloaded from: https://www.kaggle.com/c/competitive-data-science-final-project/data
item_categories =   pd.read_csv('../data/item_categories.csv')
items             = pd.read_csv('../data/items.csv')
shops             = pd.read_csv('../data/shops.csv')
sales             = pd.read_csv('../data/sales_train.csv.gz')
sample_submission = pd.read_csv('../data/sample_submission.csv.gz') # scores 1.23646 
test              = pd.read_csv('../data/test.csv.gz')

In [57]:
# remove some questionable data (see EDA file)
sales = sales[sales.item_price<100000]
sales = sales[sales.item_cnt_day<=1000]


In [58]:
# get the test set matching the format of the other data, using date_block_num = 34 
test['date_block_num'] = 34
test = pd.merge(test, items, on='item_id', how='left').drop('item_name', axis=1)
#test = test.drop('ID', axis=1)


# Add aggregates

In [59]:
# Start with a df having every item/shop combination for each date_block_num
# see: http://mlwhiz.com/blog/2017/12/26/How_to_win_a_data_science_competition/ -- via the forums
fs_data = get_all_combinations(sales)\
            .sort_values(['shop_id', 'date_block_num', 'item_id'])  # sorting makes inspecting the data easier

In [60]:
# add items df, to get item_category_id)
fs_data = pd.merge(fs_data, items, on="item_id", how="left")

In [61]:
# add aggregates at various levels
shop_item_level_aggregates     = get_shop_item_level_aggregates(sales)
shop_level_aggregates          = get_shop_level_aggregates(sales)
item_level_aggregates          = get_item_level_aggregates(sales)
item_category_level_aggregates = get_item_category_level_aggregates(pd.merge(sales, items, on='item_id'))

fs_data = pd.merge(fs_data, shop_item_level_aggregates,     on=['shop_id', 'item_id', 'date_block_num'], how='left')
fs_data = pd.merge(fs_data, shop_level_aggregates,          on=['shop_id', 'date_block_num'],            how='left')
fs_data = pd.merge(fs_data, item_level_aggregates,          on=['item_id', 'date_block_num'],            how='left')
fs_data = pd.merge(fs_data, item_category_level_aggregates, on=['item_category_id', 'date_block_num'],   how='left')

# deal with NAs from the above left joins:
# we'll assume that items not appearing in the original data sold 0
fs_data[['item_cnt_month']]  = fs_data[['item_cnt_month']].fillna(0)

# Deal with missing rows

In [62]:
# impute missing item prices
imputer = Imputer()
fs_data[['item_price']] = imputer.fit_transform(fs_data[['item_price']])
#fs_data[['item_price']] = fs_data[['item_price']].fillna(0) # alternatively replace missing prices with zero

# check we have no missing values
assert(fs_data.isnull().values.any() == False)

# Lagging

In [63]:
#lags = [1,12]
lags = [1,2,3,4,5,6]

In [64]:
# lag the data
# add test data before lagging (we added item category and date_block_num=34 above)
fs_data = pd.concat([fs_data, test], axis=0)


row_identifier_cols = ['shop_id', 'item_id', 'date_block_num']
ignore_cols         = ['item_name', 'item_category_id']
to_lag_cols         = [col for col in fs_data.columns if col not in row_identifier_cols + ignore_cols + ['ID']]

fs_data = create_lags(fs_data, 
                          lags, 
                          row_identifier_cols,
                          to_lag_cols,
                          ignore_cols)

# drop the original versions of the lagged columns, since we can't use them with the test data 
# but keep the target value
fs_data = fs_data.drop([col for col in to_lag_cols if col != 'item_cnt_month'], axis=1)

# remove rows where there isn't enough history for the lagging we've applied
fs_data = fs_data[fs_data.date_block_num > max(lags)]

# as we did before calculating lags, we'll replace counts with zero and prices with an imputed value
# (relying on column names here, which isn't ideal)
item_cnt_cols   = [col for col in fs_data.columns if 'item_cnt' in col]
item_price_cols = [col for col in fs_data.columns if 'item_price' in col]

fs_data[item_cnt_cols]   = fs_data[item_cnt_cols].fillna(0)
#fs_data[item_price_cols]  = fs_data[item_price_cols].fillna(0) # alternatively replace missing prices with zero
fs_data[item_price_cols] = imputer.fit_transform(fs_data[item_price_cols])

/home/ubuntu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:2352: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


# Clipping

In [65]:
# clip the target variables (and associated lagged versions) to 0-40 -- *not* 0-20 as per the test data)
# (following a tip from the forums)
#fs_data[item_cnt_cols] = np.clip(fs_data[item_cnt_cols], 0, 40) # replace all counts with zero
fs_data['item_cnt_month'] = np.clip(fs_data['item_cnt_month'] , 0, 40) 


# Use PCA and kmeans to get features from the text

In [66]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [67]:
item_name_vectorizer = CountVectorizer()
item_name_matrix = item_name_vectorizer.fit_transform(items.item_name).todense()

In [68]:
item_name_pca = PCA(n_components=50)
item_principal_components = item_name_pca.fit_transform(item_name_matrix)

In [69]:
item_name_km = KMeans(n_clusters=5, init='k-means++', max_iter=200, n_init=1)

In [70]:
item_name_clusters = item_name_km.fit_transform(item_principal_components)

In [71]:
items['item_name_cluster'] = item_name_km.predict(item_principal_components)

In [72]:
# are we just reproducing the item categories?
pd.crosstab(items.item_name_cluster, items.item_category_id)

item_category_id   0   1   2   3   4   5   6   7   8   9  ...  74   75   76  \
item_name_cluster                                         ...                 
0                   4   2  62  34  15  29  38  23   3  12 ...   0  268    3   
1                   0   0  13   0   0   0   2   0   0   0 ...   0    2    2   
2                   0   0   0   0   0   0   0   0   0   0 ...   0    7    0   
3                   0   0   0   0   0   0   0   0   0   0 ...   0    0    0   
4                   0   0   0   0   0   0   0   1   6   0 ...   8    0  328   

item_category_id    77   78  79  80  81  82  83  
item_name_cluster                                
0                   24    4   0   0   7   8  15  
1                    1    0   0   0   0   0   0  
2                  191   44   1   0   0   0   0  
3                    2    0   0   0   0   0   0  
4                    2  230   0   6   0   0   0  

[5 rows x 84 columns]

In [73]:
items.head()

item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  item_name_cluster  
0                40                  0  
1                76                  4  
2                40                  0  
3                40                  0  
4                40                  0

In [74]:
# join the clusters to the rest of the data
fs_data = pd.merge(fs_data, items[['item_id','item_name_cluster']], on="item_id", how="left")

# Add a flag indicating when a shop or item first appears

In [75]:
# Flag shops and items when they appear for the first time
# Theory: the model could learn to weight the lags less in these cases
shops_and_items = fs_data[['shop_id', 'item_id', 'date_block_num']]

first_shop_appearance = shops_and_items\
                        .drop(['item_id'], axis=1)\
                        .sort_values(by=['shop_id', 'date_block_num'], ascending=[True, True])\
                        .groupby('shop_id')\
                        .head(1)\
                        .rename(columns={'date_block_num' : 'shop_id_first_appearance'})

first_item_appearance = shops_and_items\
                        .drop(['shop_id'], axis=1)\
                        .sort_values(by=['item_id', 'date_block_num'], ascending=[True, True])\
                        .groupby('item_id')\
                        .head(1)\
                        .rename(columns={'date_block_num' : 'item_id_first_appearance'})

# use the first appearance to add a 1/0 flag indicating whether we've seen the item before, then drop the original columns
fs_data = pd.merge(fs_data, first_shop_appearance, on='shop_id', how='inner')
fs_data['shop_already_seen'] = np.where(fs_data['shop_id_first_appearance'] < fs_data['date_block_num'], 1, 0 )

fs_data = pd.merge(fs_data, first_item_appearance, on='item_id', how='inner')
fs_data['item_already_seen'] = np.where(fs_data['item_id_first_appearance'] < fs_data['date_block_num'], 1, 0 )

fs_data = fs_data.drop(['shop_id_first_appearance', 'item_id_first_appearance'], axis=1)

# Add interactions / trends

## lags 1 to 6 only

In [76]:
# (different versions of this for diff lag schemes
fs_data['item_cnt_month_lag1_lag2_diff']  = fs_data['item_cnt_month_lag1'] - fs_data['item_cnt_month_lag2']

fs_data['item_cnt_month_lag2_lag3_diff']  = fs_data['item_cnt_month_lag2'] - fs_data['item_cnt_month_lag3']

fs_data['item_cnt_month_lag1+2_lag3+3_diff']  = (fs_data['item_cnt_month_lag1'] + fs_data['item_cnt_month_lag2']) - \
        (fs_data['item_cnt_month_lag3'] + fs_data['item_cnt_month_lag4'])

fs_data['item_cnt_month_lag1sq_lag2_diff']  = fs_data['item_cnt_month_lag1']**2 - fs_data['item_cnt_month_lag2']


In [77]:
fs_data['ITEM_mean_item_cnt_day_lag1_lag2_diff']  = fs_data['ITEM_mean_item_cnt_day_lag1'] - fs_data['ITEM_mean_item_cnt_day_lag2']

fs_data['ITEM_mean_item_cnt_day_lag2_lag3_diff']  = fs_data['ITEM_mean_item_cnt_day_lag2'] - fs_data['ITEM_mean_item_cnt_day_lag3']

fs_data['ITEM_mean_item_cnt_day_lag1+2_lag3+3_diff']  = (fs_data['ITEM_mean_item_cnt_day_lag1'] + fs_data['ITEM_mean_item_cnt_day_lag2']) - \
        (fs_data['ITEM_mean_item_cnt_day_lag3'] + fs_data['ITEM_mean_item_cnt_day_lag4'])

fs_data['ITEM_mean_item_cnt_day_lag1sq_lag2_diff']  = fs_data['ITEM_mean_item_cnt_day_lag1']**2 - fs_data['ITEM_mean_item_cnt_day_lag2']


In [78]:
fs_data['SHOP_mean_item_cnt_day_lag1_lag2_diff']  = fs_data['SHOP_mean_item_cnt_day_lag1'] - fs_data['SHOP_mean_item_cnt_day_lag2']

fs_data['SHOP_mean_item_cnt_day_lag2_lag3_diff']  = fs_data['SHOP_mean_item_cnt_day_lag2'] - fs_data['SHOP_mean_item_cnt_day_lag3']

fs_data['SHOP_mean_item_cnt_day_lag1+2_lag3+3_diff']  = (fs_data['SHOP_mean_item_cnt_day_lag1'] + fs_data['SHOP_mean_item_cnt_day_lag2']) - \
        (fs_data['SHOP_mean_item_cnt_day_lag3'] + fs_data['SHOP_mean_item_cnt_day_lag4'])

fs_data['SHOP_mean_item_cnt_day_lag1sq_lag2_diff']  = fs_data['SHOP_mean_item_cnt_day_lag1']**2 - fs_data['SHOP_mean_item_cnt_day_lag2']


## lags 1 and 12 only

In [24]:
fs_data['item_cnt_month_lag1_lag12_diff']  = fs_data['item_cnt_month_lag1'] - fs_data['item_cnt_month_lag12']


In [25]:
fs_data['ITEM_mean_item_cnt_day_lag1_lag12_diff']  = fs_data['ITEM_mean_item_cnt_day_lag1'] - fs_data['ITEM_mean_item_cnt_day_lag12']

In [26]:
fs_data['SHOP_mean_item_cnt_day_lag1_lag12_diff']  = fs_data['SHOP_mean_item_cnt_day_lag1'] - fs_data['SHOP_mean_item_cnt_day_lag12']

In [27]:
fs_data['item_cnt_month_lag1sq_lag12_diff']  = fs_data['item_cnt_month_lag1']**2 - fs_data['item_cnt_month_lag12']


# Save the processed data

In [79]:
#fs_data.to_pickle('../data/fs_data_lags_1_to_6.pkl')
fs_data.to_pickle('../data/fs_data_lags_1_12.pkl')

In [80]:
fs_data.columns

Index(['ID', 'date_block_num', 'item_category_id', 'item_cnt_month', 'item_id',
       'item_name', 'shop_id', 'ITEM_CATEGORY_mean_item_cnt_day_lag1',
       'ITEM_CATEGORY_mean_item_price_lag1',
       'ITEM_CATEGORY_sum_item_cnt_day_lag1', 'ITEM_mean_item_cnt_day_lag1',
       'ITEM_mean_item_price_lag1', 'ITEM_sum_item_cnt_day_lag1',
       'SHOP_mean_item_cnt_day_lag1', 'SHOP_mean_item_price_lag1',
       'SHOP_sum_item_cnt_day_lag1', 'item_cnt_month_lag1', 'item_price_lag1',
       'ITEM_CATEGORY_mean_item_cnt_day_lag2',
       'ITEM_CATEGORY_mean_item_price_lag2',
       'ITEM_CATEGORY_sum_item_cnt_day_lag2', 'ITEM_mean_item_cnt_day_lag2',
       'ITEM_mean_item_price_lag2', 'ITEM_sum_item_cnt_day_lag2',
       'SHOP_mean_item_cnt_day_lag2', 'SHOP_mean_item_price_lag2',
       'SHOP_sum_item_cnt_day_lag2', 'item_cnt_month_lag2', 'item_price_lag2',
       'ITEM_CATEGORY_mean_item_cnt_day_lag3',
       'ITEM_CATEGORY_mean_item_price_lag3',
       'ITEM_CATEGORY_sum_item_cnt_day_lag